# Data Cleaning

In [15]:
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
from uszipcode import SearchEngine
import numpy as np
from progressbar import ProgressBar
import math
import json

from sklearn.model_selection import train_test_split
# state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [2]:
### NOTE: You must unzip the 500 Cities Dataset first before reading it in with this line
raw_health = pd.read_csv("./sourcedata/500_Cities__Local_Data_for_Better_Health__2017_release.csv")
raw_metro_income = pd.read_csv('./sourcedata/metropolitan_income_data.csv')
raw_county_income = pd.read_csv("./sourcedata/county_income.csv")

### Clean County Income Data

In [ ]:
raw_county_income.dropna(how="all",inplace=True)
raw_county_income.dropna(subset=["2017"], inplace=True)

In [ ]:
raw_county_income["Rank_In_State_2018"] = raw_county_income["Rank_In_State_2018"].astype(int)

In [ ]:
raw_county_income.reset_index(drop=True, inplace=True)
raw_county_income.head()

In [ ]:
#Need to reformat raw_county_income which has both states and counties listed in rows
counties_dicts = []
for i in range(raw_county_income.shape[0]):
    if raw_county_income["Rank_In_State_2018"][i] == 0: #if ==0, it is the state, not a county
        state_name = raw_county_income["County_Or_State"][i]
    else:
        county_dict = {"city_county":raw_county_income["County_Or_State"][i],
                       "state":state_name,
                       "2017":raw_county_income["2017"][i],
                      "Rank_In_State_2018":raw_county_income["Rank_In_State_2018"][i]}
        counties_dicts.append(county_dict)
    if(i == raw_county_income.shape[0]-1) or (raw_county_income["Rank_In_State_2018"][i+1] == 0):
        counties_dicts.append(county_dict)
    
        

In [ ]:
county_income_df = pd.DataFrame(counties_dicts, index=range(len(counties_dicts)))
county_income_df["type"] = ["county" for i in range(county_income_df.shape[0])]

In [ ]:
county_income_df.head()

### Clean Metro (City) Income Data

In [ ]:
raw_metro_income.head(2)

In [ ]:
raw_metro_income["city_county"] = [i.split(",")[0] for i in raw_metro_income["Metro_Area"]]
raw_metro_income["state"] = [i.split(",")[1].strip() for i in raw_metro_income["Metro_Area"]]
raw_metro_income.drop(["Metro_Area","2016","2018"], axis = 1, inplace = True)

In [ ]:
raw_metro_income["type"] = ["city" for i in range(raw_metro_income.shape[0])]
raw_metro_income.head()

### Combine County and Metro(City) Income DataFrames

In [ ]:
income = pd.concat([raw_metro_income, county_income_df])

In [ ]:
income.reset_index(drop=True, inplace=True)
print(income.shape)
income.head()

In [ ]:
# Get Zipcodes for income data by city and state
def get_zipcode_by_city_state(city:str, state:str, search_engine=None) -> str:
    if search_engine == None:
        search_engine = SearchEngine(simple_zipcode=True)
    try:
        zips = search_engine.by_city_and_state(city, state)
        return zips.zipcode
    except AttributeError:
        return ",".join([z.zipcode for z in zips]) 
    except KeyError:
        return "NotFound"

In [ ]:
search_engine = SearchEngine(simple_zipcode=True)
zipcodes = []
for row in range(income.shape[0]):
    zipcode = get_zipcode_by_city_state(income["city_county"][row], income["state"][row], search_engine)
    zipcodes.append(zipcode)
        

In [ ]:
income["zips"] = zipcodes

In [ ]:
income.to_csv("./processeddata/income_city_county_zipcode.csv")

### Clean Health Data

In [3]:
raw_health["GeographicLevel"].value_counts()

Census Tract    782047
City             28000
US                  56
Name: GeographicLevel, dtype: int64

In [4]:
# states = raw_health[raw_health["GeographicLevel"] == "US"]
# cities = raw_health[raw_health["GeographicLevel"] == "City"]
census_tracts = raw_health[raw_health["GeographicLevel"] == "Census Tract"]
census_tracts.reset_index(drop = True, inplace = True)

In [5]:
census_tracts.columns

Index(['Year', 'StateAbbr', 'StateDesc', 'CityName', 'GeographicLevel',
       'DataSource', 'Category', 'UniqueID', 'Measure', 'Data_Value_Unit',
       'DataValueTypeID', 'Data_Value_Type', 'Data_Value',
       'Low_Confidence_Limit', 'High_Confidence_Limit',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'PopulationCount',
       'GeoLocation', 'CategoryID', 'MeasureId', 'CityFIPS', 'TractFIPS',
       'Short_Question_Text'],
      dtype='object')

In [6]:
census_tracts.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2015,AL,Alabama,Birmingham,Census Tract,BRFSS,Prevention,0107000-01073000100,Current lack of health insurance among adults ...,%,...,27.2,NaN,NaN,3042.0,"(33.5794328326, -86.7228323926)",PREVENT,ACCESS2,107000.0,1.073000e+09,Health Insurance
1,2015,AL,Alabama,Birmingham,Census Tract,BRFSS,Prevention,0107000-01073000300,Current lack of health insurance among adults ...,%,...,32.4,NaN,NaN,2735.0,"(33.5428208686, -86.752433978)",PREVENT,ACCESS2,107000.0,1.073000e+09,Health Insurance
2,2015,AL,Alabama,Birmingham,Census Tract,BRFSS,Prevention,0107000-01073000400,Current lack of health insurance among adults ...,%,...,29.9,NaN,NaN,3338.0,"(33.5632449633, -86.7640474064)",PREVENT,ACCESS2,107000.0,1.073000e+09,Health Insurance
3,2015,AL,Alabama,Birmingham,Census Tract,BRFSS,Prevention,0107000-01073000500,Current lack of health insurance among adults ...,%,...,32.0,NaN,NaN,2864.0,"(33.5442404594, -86.7749130719)",PREVENT,ACCESS2,107000.0,1.073000e+09,Health Insurance
4,2015,AL,Alabama,Birmingham,Census Tract,BRFSS,Prevention,0107000-01073000700,Current lack of health insurance among adults ...,%,...,36.7,NaN,NaN,2577.0,"(33.5525406139, -86.8016893706)",PREVENT,ACCESS2,107000.0,1.073001e+09,Health Insurance


In [7]:
# Example of a Geolocation from the Data
census_tracts.GeoLocation[0]

'(33.5794328326, -86.7228323926)'

In [8]:
#Create latitude and longitude cols from GeoLocation
latitudes = []
longitudes = []
for row in census_tracts["GeoLocation"]:
    swap = str(row)
    swap = swap.replace("(","").replace(")","").split(",")
    lat, long = swap[0], swap[1].strip()
    latitudes.append(lat)
    longitudes.append(long)
census_tracts["latitude"] = latitudes
census_tracts["longitude"] = longitudes

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
# Health data is structured odd, need to create a list of the various measures tracked in the dataset
measures = list(census_tracts.Measure.value_counts().index)

In [10]:
# These are each of the measures in the data, will have to reformat the data
print(measures)

['Taking medicine for high blood pressure control among adults aged >=18 Years with high blood pressure', 'Mental health not good for >=14 days among adults aged >=18 Years', 'Diagnosed diabetes among adults aged >=18 Years', 'Obesity among adults aged >=18 Years', 'Visits to doctor for routine checkup within the past Year among adults aged >=18 Years', 'Chronic kidney disease among adults aged >=18 Years', 'Current asthma among adults aged >=18 Years', 'Current smoking among adults aged >=18 Years', 'Physical health not good for >=14 days among adults aged >=18 Years', 'Binge drinking among adults aged >=18 Years', 'Stroke among adults aged >=18 Years', 'Cholesterol screening among adults aged >=18 Years', 'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years', 'Arthritis among adults aged >=18 Years', 'Visits to dentist or dental clinic among adults aged >=18 Years', 'Sleeping less than 7 hours among adults aged >=18 Years', 'No leisure-time physic

In [11]:
### Create a dictionary to link metric abbreviations to meanings
measure_abbvs = list(census_tracts.MeasureId.value_counts().index)

In [13]:
measure_dictionary = {}
for abbv in measure_abbvs:
    meaning = census_tracts.loc[census_tracts["MeasureId"] == abbv].head(1)["Measure"].values[0]
    measure_dictionary[abbv] = meaning

In [19]:
## Write the measure meanings dictionary to a json file for use during modeling and exploration
with open("./processeddata/measure_meanings.json", "w+") as f:
    f.write(json.dumps(measure_dictionary,indent=4))

In [ ]:
# making a separate df for each measure
census_measures_dfs = [census_tracts[census_tracts["Measure"] == measure] for measure in measures] 

In [ ]:
census_measures_dfs[0].head(2)

In [ ]:
# A little bit of reformatting of our dataframes, which will be merged together into a master df
metric_dfs = []
for df in census_measures_dfs:
    df.reset_index(inplace=True, drop=True)
    temp = pd.DataFrame({df.MeasureId[0]:df.Data_Value, # this is the actual value for the measure
                         "UniqueID":df.UniqueID,"geometry":df.GeoLocation, # UniqueID and geolocation cols
                         "state":df.StateAbbr, "city_county":df.CityName,
                         "longitude": df.longitude, "latitude":df.latitude,
                        "tract_pop_count":df.PopulationCount})
    metric_dfs.append(temp)

In [ ]:
metric_dfs[2].head()

In [ ]:
# Create master DF of all metrics dfs
census_health = pd.merge(metric_dfs[0],metric_dfs[1], how='left',on="UniqueID") 
for i in metric_dfs[2:]: # merging the rest with a for loop
    census_health = census_health.merge(i, how = "left",on="UniqueID")
census_health.reset_index(drop=True,inplace=True)

In [ ]:
cols = list(census_health.columns)
for col in cols:
    if col.endswith("_y"):
        census_health.drop(col, axis = 1, inplace = True)
    elif col.endswith("_x"):
        census_health.rename({col:col.replace("_x","")}, axis=1, inplace = True)

census_health = census_health.loc[:,~census_health.columns.duplicated()] # remove duplicates


In [ ]:
census_health.head()

In [ ]:
## Add zip codes to census tracts based
def get_zipcode_by_lat_long(lat:str, long:str, radius=25, search_engine=None) -> str:
    if search_engine == None:
        search_engine = SearchEngine(simple_zipcode=True)
    try:
        zips = search_engine.by_coordinates(float(lat), float(long),radius, returns=1)#only taking best hit on zip, lat long should be precise enough for 1 zip
        return zips.zipcode
    except AttributeError:
        return ",".join([z.zipcode for z in zips]) 
    except KeyError:
        return "NotFound"

In [ ]:
search_engine = SearchEngine(simple_zipcode=True)
zipcodes = []
for row in range(census_health.shape[0]):
    zipcode = get_zipcode_by_lat_long(census_health["latitude"][row], census_health["longitude"][row], search_engine = search_engine)
    zipcodes.append(zipcode)

In [ ]:
census_health["zips"] = zipcodes

In [ ]:
census_health.to_csv("./processeddata/census_health_data_zipcode.csv")

### Merge Health And Income Data On Zip Code

In [ ]:
income = pd.read_csv("./processeddata/income_city_county_zipcode.csv", dtype = {"zips":str})
census_health = pd.read_csv("./processeddata/census_health_data_zipcode.csv", dtype = {"zips":str})

In [ ]:
income.drop(["Unnamed: 0","city_county","state"], axis=1, inplace=True)
income.dropna(subset=["zips"], inplace=True)

census_health.drop("Unnamed: 0", axis = 1, inplace = True)
census_health.reset_index(drop=True, inplace=True)

In [ ]:
income.columns

In [ ]:
#Zipcodes not identified for 1037 counties/cities
income["zips"].isna().sum(), income.shape

In [ ]:
census_health.head()

In [ ]:
health_zipcodes = [str(i) for i in census_health["zips"].value_counts().index]

In [ ]:
bar = ProgressBar()

# Create income lookup dict from income df
median_income_2017 = {}
for zipcode in bar(census_health["zips"].value_counts().index):

    filtered_df = income[income["zips"].str.contains(str(zipcode))] 
    
    swap_income = [int(val) for val in filtered_df["2017"].values if not math.isnan(val)] #might hit on multiple zips
    mean_of_median_incs = np.mean(swap_income)
    median_income_2017[str(zipcode)] = mean_of_median_incs
        

In [ ]:
bar = ProgressBar()

census_health["Median_Income_2017"] = [np.NaN for i in range(census_health.shape[0])]
for zipcode, inc in bar(median_income_2017.items()):
    census_health.loc[census_health["zips"] == zipcode, "Median_Income_2017"] = inc

In [ ]:
# Need more precise income data as a lot of zipcodes don't have income data, may want to impute based on closest locs
census_health.Median_Income_2017.isna().sum(), census_health.shape

In [ ]:
census_health.to_csv("./processeddata/census_health_citycounty_income_final.csv")